<a href="https://colab.research.google.com/github/Srestrero/AlgorithmsUN2024II/blob/main/LAB_ATQ/srestreporo_LAB_ATQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#   Q18 Machine Learning Rolling Basis
#   Santiago Restrepo Rojas

In this example we predict whether the price will rise or fall by using supervised learning (Random Forest Regressor). This template represents a starting point for developing a system which can take part to the Q18 S&P 500 Stock Long-Short contest.

It consists of two parts.

* In the **first part** we just perform a global training of the time series using all time series data. We disregard the sequential aspect of the data and use also future data to train past data.

* In the **second part** we use the built-in backtester and perform training and prediction on a rolling basis in order to avoid forward looking. Please note that we are using a **specialized** version of the Quantiacs backtester which dramatically speeds up the the backtesting process by retraining your model on a regular basis.

**Features for learning**: we will use several technical indicators trying to capture different features. You can have a look at [**Technical Indicators**](https://quantiacs.com/documentation/en/user_guide/technical_indicators.html).

Please note that:

* Your trading algorithm can open short and long positions.

* At each point in time your algorithm can trade all or a subset of the stocks which at that point of time are or were part of the NASDAQ-100 stock index. Note that the composition of this set changes in time, and Quantiacs provides you with an appropriate filter function for selecting them.

* The Sharpe ratio of your system since January 1st, 2006, has to be larger than 1.

* Your system cannot be a copy of the current examples. We run a correlation filter on the submissions and detect duplicates.

* For simplicity we will use a single asset. It pays off to use more assets, ideally uncorrelated, and diversify your positions for a more solid Sharpe ratio.

More details on the rules can be found [here](https://quantiacs.com/contest).

**Need help?** Check the [**Documentation**](https://quantiacs.com/documentation/en/) and find solutions/report problems in the [**Forum**](https://quantiacs.com/community/categories) section.

**More help with Jupyter?** Check the official [**Jupyter**](https://jupyter.org/) page.

Once you are done, click on **Submit to the contest** and take part to our competitions.

API reference:

* **data**: check how to work with [data](https://quantiacs.com/documentation/en/reference/data_load_functions.html);

* **backtesting**: read how to run the [simulation](https://quantiacs.com/documentation/en/reference/evaluation.html) and check the results.

Need to use the optimizer function to automate tedious tasks?

* **optimization**: read more on our [article](https://quantiacs.com/community/topic/29/optimizing-and-monitoring-a-trading-system-with-quantiacs).

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

<IPython.core.display.Javascript object>

In [ ]:
import xarray as xr
import numpy as np
import qnt.stats as qnstats
import qnt.data as qndata
import qnt.output as qnout
import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.graph as qngraph
import qnt.exposure as qnexp
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import logging

In [ ]:
# Cargar datos del S&P 500
snp_stocks_data = qndata.stocks_load_spx_data(min_date='2005-06-01')
# Seleccionar los 20 activos más líquidos
vol = snp_stocks_data.sel(field="vol")
liq = snp_stocks_data.sel(field="is_liquid")
close = snp_stocks_data.sel(field="close")
money_vol = vol * liq * close
total_money_vol = money_vol.sum(dim='time', skipna=True)
top_20_assets = total_money_vol.to_pandas().nlargest(20).index.tolist()
stock_data = snp_stocks_data.sel(asset=top_20_assets)

In [ ]:
def get_features(data):
    """Calculate technical indicators as features"""
    # Precios y volumen
    close = data.sel(field="close")
    high = data.sel(field="high")
    low = data.sel(field="low")
    vol = data.sel(field="vol")
    liq = data.sel(field="is_liquid")

    # Indicadores técnicos
    atrs = qnta.atr(high=high, low=low, close=close, ma=14)
    sma_20 = qnta.sma(close, 20)
    sma_50 = qnta.sma(close, 50)

    # Money flow indicators
    money_vol = vol * liq * close
    total_money_vol = money_vol.sum(dim='asset', skipna=True)
    money_vol_share = money_vol / total_money_vol
    mvs_mov = qnta.sma(money_vol_share, 135)

    # Ratio ATR/Close como en el ejemplo original
    ratio = atrs / close

    features = xr.concat([
        sma_20.assign_coords(field="sma20"),
        sma_50.assign_coords(field="sma50"),
        atrs.assign_coords(field="atr"),
        ratio.assign_coords(field="atr_ratio"),
        money_vol_share.assign_coords(field="money_vol_share"),
        mvs_mov.assign_coords(field="mvs_mov")
    ], dim="field")

    return features

In [ ]:
# Calcular features
my_features = get_features(stock_data)

# Mostrar un resumen de los features
print("Dimensiones de los features:", my_features.dims)
print("\nPrimeros valores para un activo específico:")
print(my_features.isel(asset=0).to_pandas().head())
print("\nActivos disponibles:", my_features.asset.values)

In [ ]:
def get_target_classes(data):
    """Define target values for ML prediction"""
    close = data.sel(field="close")
    future_close = qnta.shift(close, -1)  # Precio del siguiente día

    # Calcular retorno porcentual manualmente
    future_return = (future_close - close) / close

    # Clasificación multi-clase basada en los retornos
    target = xr.where(future_return > 0.02, 2,  # Strong up
             xr.where(future_return > 0, 1,     # Moderate up
             xr.where(future_return > -0.02, 0, # Sideways
             -1)))                              # Down

    return target

In [ ]:
# Calcular targets
my_targetclass = get_target_classes(stock_data)

# Mostrar un resumen de los targets
print("Dimensiones de los targets:", my_targetclass.dims)
print("\nPrimeros valores para un activo específico:")
print(my_targetclass.isel(asset=0).to_pandas().head())

In [ ]:
def get_model():
    """Constructor for Random Forest model"""
    return RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42
    )

In [ ]:
def train_models(features, targets):
    """Train models for each asset"""
    asset_name_all = features.coords["asset"].values
    models = {}
    scaler = StandardScaler()

    for asset_name in asset_name_all:
        try:
            # Preparar datos
            target_cur = targets.sel(asset=asset_name).dropna(dim="time", how="any")
            features_cur = features.sel(asset=asset_name).dropna(dim="time", how="any")

            # Alinear features y targets
            target_aligned, features_aligned = xr.align(target_cur, features_cur, join="inner")

            if len(features_aligned.time) < 10:
                logging.warning(f"Insufficient data for {asset_name}")
                continue

            # Preparar datos para el modelo
            features_array = features_aligned.transpose("time", "field").values
            target_array = target_aligned.values

            # Verificar que tenemos datos válidos
            if len(features_array) != len(target_array):
                logging.warning(f"Data mismatch for {asset_name}")
                continue

            # Escalado de características
            features_scaled = scaler.fit_transform(features_array)

            # Entrenar modelo
            model = get_model()
            model.fit(features_scaled, target_array)
            models[asset_name] = {
                'model': model,
                'scaler': scaler
            }

            print(f"Successfully trained model for {asset_name}")

        except Exception as e:
            logging.exception(f"Error training model for {asset_name}: {str(e)}")

    return models

# Entrenar modelos
print("Iniciando entrenamiento de modelos...")
models = train_models(my_features, my_targetclass)
print(f"Modelos entrenados: {len(models)}")

In [ ]:
def plot_feature_importance():
    """Visualiza la importancia de las características para cada modelo"""
    for asset_name, model_dict in models.items():
        print(f"\nImportancia de características para {asset_name}:")

        # Obtener importancia de características del Random Forest
        importance = model_dict['model'].feature_importances_

        # Obtener nombres de las características
        feature_names = list(my_features.field.values)

        # Crear DataFrame para mejor visualización
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)

        # Mostrar valores
        for idx, row in importance_df.iterrows():
            print(f"{row['feature']}: {row['importance']:.5f}")

        # Visualización
        plt.figure(figsize=(10, 6))
        plt.bar(importance_df['feature'], importance_df['importance'])
        plt.title(f'Feature Importance for {asset_name}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

# Mostrar importancia de características
plot_feature_importance()

In [ ]:
def generate_weights(data):
    """Genera pesos basados en las predicciones del modelo"""
    # Inicializar pesos
    weights = xr.zeros_like(data.sel(field="close"))

    # Calcular features para predicción
    features = get_features(data)

    for asset_name, model_dict in models.items():
        try:
            # Obtener y preparar features
            features_cur = features.sel(asset=asset_name).dropna(dim="time", how="any")

            if len(features_cur.time) < 1:
                print(f"No hay suficientes datos para {asset_name}")
                continue

            # Preparar datos para predicción
            features_array = features_cur.transpose("time", "field").values

            # Escalar features
            features_scaled = model_dict['scaler'].transform(features_array)

            # Realizar predicción
            predictions = model_dict['model'].predict(features_scaled)

            # Asignar pesos
            weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = predictions

        except Exception as e:
            logging.exception(f"Error en predicción para {asset_name}: {str(e)}")

    return weights

# Generar pesos
print("Generando predicciones y pesos...")
weights = generate_weights(stock_data)
print("\nResumen de pesos generados:")
print(f"Shape: {weights.shape}")
print("\nEstadísticas descriptivas:")
print(weights.to_pandas().describe())

In [ ]:
def calculate_metrics(data, weights):
    """Calcula múltiples métricas de rendimiento"""
    try:
        # Calcular retornos relativos
        returns = qnstats.calc_relative_return(data, weights)

        # Calcular Sharpe Ratio
        sharpe = qnstats.calc_sharpe_ratio_annualized(returns).values[-1]

        # Calcular Sortino Ratio
        sortino = qnstats.calc_sortino_ratio(returns).values[-1]

        # Calcular máximo drawdown
        max_dd = qnstats.calc_max_drawdown(returns).values[-1]

        print("\nMétricas de rendimiento:")
        print(f"Sharpe Ratio: {sharpe:.2f}")
        print(f"Sortino Ratio: {sortino:.2f}")
        print(f"Maximum Drawdown: {max_dd:.2%}")

        return {
            'sharpe': sharpe,
            'sortino': sortino,
            'max_drawdown': max_dd
        }

    except Exception as e:
        logging.exception("Error calculando métricas")
        return None

# Calcular y mostrar métricas
metrics = calculate_metrics(stock_data, weights)
if metrics:
    # Visualizar retornos acumulados
    returns = qnstats.calc_relative_return(stock_data, weights)
    cumulative_returns = (1 + returns).cumprod()

    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_returns.time, cumulative_returns)
    plt.title('Retornos Acumulados')
    plt.xlabel('Tiempo')
    plt.ylabel('Retorno Acumulado')
    plt.yscale('log')
    plt.grid(True)
    plt.show()

In [ ]:
qnout.write(weights)

In [ ]:
def calculate_portfolio_performance(data, weights, initial_capital=1_000_000):
    """Calcula el rendimiento del portafolio y la contribución por activo"""
    try:
        # Calcular retornos relativos
        returns = qnstats.calc_relative_return(data, weights)

        # Calcular equity curve (valor del portafolio en el tiempo)
        portfolio_value = initial_capital * (1 + returns).cumprod()

        # Calcular retornos por activo
        asset_returns = {}
        final_values = {}
        contributions = {}

        for asset in weights.asset.values:
            # Crear pesos solo para este activo
            single_asset_weights = xr.zeros_like(weights)
            single_asset_weights.loc[dict(asset=asset)] = weights.sel(asset=asset)

            # Calcular retornos para este activo
            asset_return = qnstats.calc_relative_return(data, single_asset_weights)
            asset_value = initial_capital * (1 + asset_return).cumprod()

            asset_returns[asset] = asset_return
            final_values[asset] = float(asset_value.isel(time=-1))
            contributions[asset] = final_values[asset] - initial_capital

        # Ordenar contribuciones por valor
        sorted_contributions = dict(sorted(contributions.items(), key=lambda x: x[1], reverse=True))

        # Mostrar resultados
        final_portfolio_value = float(portfolio_value.isel(time=-1))
        total_return = ((final_portfolio_value - initial_capital) / initial_capital) * 100

        print(f"\nAnálisis de Portafolio con Capital Inicial: ${initial_capital:,.2f}")
        print("-" * 60)
        print(f"Valor Final del Portafolio: ${final_portfolio_value:,.2f}")
        print(f"Retorno Total: {total_return:.2f}%")

        print("\nContribución por Activo:")
        print("-" * 60)
        for asset, contribution in sorted_contributions.items():
            pct_contribution = (contribution / (final_portfolio_value - initial_capital)) * 100
            print(f"{asset}:")
            print(f"  Contribución: ${contribution:,.2f}")
            print(f"  % del Retorno Total: {pct_contribution:.2f}%")

        # Visualización
        plt.figure(figsize=(15, 6))
        plt.bar(sorted_contributions.keys(), sorted_contributions.values())
        plt.title('Contribución por Activo al Retorno Total')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Contribución ($)')
        plt.tight_layout()
        plt.show()

        # Gráfica de evolución del valor del portafolio
        plt.figure(figsize=(15, 6))
        plt.plot(portfolio_value.time, portfolio_value)
        plt.title('Evolución del Valor del Portafolio')
        plt.xlabel('Tiempo')
        plt.ylabel('Valor ($)')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        return {
            'final_value': final_portfolio_value,
            'total_return': total_return,
            'contributions': sorted_contributions
        }

    except Exception as e:
        logging.exception("Error en el cálculo del rendimiento del portafolio")
        return None

# Calcular y mostrar el rendimiento del portafolio
portfolio_analysis = calculate_portfolio_performance(stock_data, weights, initial_capital=1_000_000)

The sharpe ratio using the method above follows from **forward looking**. Predictions for (let us say) 2017 know about the relation between features and targets in 2020. Let us visualize the results:

In [ ]:
import qnt.graph as qngraph

statistics = qnstats.calc_stat(stock_data, weights)

display(statistics.to_pandas().tail())

performance = statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())

# check for correlations with existing strategies:
qnstats.print_correlation(weights,stock_data)

In [ ]:
"""R2 (coefficient of determination) regression score function."""
r2_score(my_targetclass, weights, multioutput="variance_weighted")

In [ ]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
explained_variance_score(my_targetclass, weights, multioutput="uniform_average")

In [ ]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
mean_absolute_error(my_targetclass, weights)

Let us now use the Quantiacs **backtester** for avoiding **forward looking**.

The backtester performs some transformations: it trains the model on one slice of data (using only data from the past) and predicts the weights for the following slice on a rolling basis:

In [ ]:
def train_model(data):
    """Create and train the model working on an asset-by-asset basis."""

    asset_name_all = data.coords["asset"].values
    features_all   = get_features(data)
    target_all     = get_target_classes(data)

    models = dict()

    for asset_name in asset_name_all:

        # drop missing values:
        target_cur   = target_all.sel(asset=asset_name).dropna("time", "any")
        features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

        target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

        if len(features_cur.time) < 10:
                continue

        model = get_model()

        try:
            model.fit(feature_for_learn_df.values, target_for_learn_df)
            models[asset_name] = model

        except:
            logging.exception("model training failed")

    return models

In [ ]:
def predict_weights(models, data):
    """The model predicts if the price is going up or down.
       The prediction is performed for several days in order to speed up the evaluation."""

    asset_name_all = data.coords["asset"].values
    weights = xr.zeros_like(data.sel(field="close"))

    for asset_name in asset_name_all:
        if asset_name in models:
            model = models[asset_name]
            features_all = get_features(data)
            features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

            if len(features_cur.time) < 1:
                continue

            try:
                weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)

            except KeyboardInterrupt as e:
                raise e

            except:
                logging.exception("model prediction failed")

    return weights

In [ ]:
# Calculate weights using the backtester:
weights = qnbt.backtest_ml(
    train                         = train_model,
    predict                       = predict_weights,
    train_period                  =  2 *365,  # the data length for training in calendar days
    retrain_interval              = 10 *365,  # how often we have to retrain models (calendar days)
    retrain_interval_after_submit = 1,        # how often retrain models after submission during evaluation (calendar days)
    predict_each_day              = False,    # Is it necessary to call prediction for every day during backtesting?
                                              # Set it to True if you suspect that get_features is looking forward.
    competition_type              = "stocks_nasdaq100",  # competition type
    lookback_period               = 365,                 # how many calendar days are needed by the predict function to generate the output
    start_date                    = "2005-01-01",        # backtest start date
    analyze                       = True,
    build_plots                   = True  # do you need the chart?
)

The Sharpe ratio is obviously smaller as the training process is not looking forward (as it happens by processing data on a global basis), but performed on a rolling basis.

# May I import libraries?

Yes, please refer to the file **init.ipynb** in your home directory. You can for example use:

! conda install -y scikit-learn

# How to load data?

Daily stock data for the **Q18 Nasdaq-100** contest can be loaded using:
```python
data = qndata.stocks.load_ndx_data(tail = 17*365, dims = ("time", "field", "asset"))
```

Cryptocurrency daily data used for the Q16/Q17 contests can be loaded using:
```python
data = qndata.cryptodaily.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

Futures data for the Q15 contest can be loaded using:
```python
data= qndata.futures.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

BTC Futures data for the Q15 contest can be loaded using:
```python
data= qndata.cryptofutures.load_data(tail = 17*365, dims = ("time", "field", "asset"))
```

# How to view a list of all tickers?

```python
data.asset.to_pandas().to_list()
```

# How to see which fields are available?

```python
data.field.to_pandas().to_list()
```

# How to load specific tickers?

```python
data = qndata.stocks.load_ndx_data(tail=17 * 365, assets=["NAS:AAPL", "NAS:AMZN"])
```

# How to select specific tickers after loading all data?

```python
def get_data_filter(data, assets):
    filler= data.sel(asset=assets)
    return filler

get_data_filter(data, ["NAS:AAPL", "NAS:AMZN"])
```

# How to get the prices for the previous day?

```python
qnta.shift(data.sel(field="open"), periods=1)
```

or:

```python
data.sel(field="open").shift(time=1)
```

# How to get the Sharpe ratio?

```python
import qnt.stats as qnstats

def get_sharpe(market_data, weights):
    rr = qnstats.calc_relative_return(market_data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(data, weights) # weights.sel(time=slice("2006-01-01",None))
```

# How do I get a list of the top 3 assets ranked by Sharpe ratio?

```python
import qnt.stats as qnstats

data = qndata.stocks.load_ndx_data(tail = 17*365, dims = ("time", "field", "asset"))

def get_best_instruments(data, weights, top_size):
    # compute statistics:
    stats_per_asset = qnstats.calc_stat(data, weights, per_asset=True)
    # calculate ranks of assets by "sharpe_ratio":
    ranks = (-stats_per_asset.sel(field="sharpe_ratio")).rank("asset")
    # select top assets by rank "top_period" days ago:
    top_period = 1
    rank = ranks.isel(time=-top_period)
    top = rank.where(rank <= top_size).dropna("asset").asset

    # select top stats:
    top_stats = stats_per_asset.sel(asset=top.values)

    # print results:
    print("SR tail of the top assets:")
    display(top_stats.sel(field="sharpe_ratio").to_pandas().tail())

    print("avg SR = ", top_stats[-top_period:].sel(field="sharpe_ratio").mean("asset")[-1].item())
    display(top_stats)
    return top_stats.coords["asset"].values

get_best_instruments(data, weights, 3)
```

# How can I check the results for only the top 3 assets ranked by Sharpe ratio?

Select the top assets and then load their data:

```python
best_assets= get_best_instruments(data, weights, 3)

data= qndata.stocks.load_ndx_data(tail = 17*365, assets=best_assets)
```

# How can prices be processed?

Simply import standard libraries, for example **numpy**:

```python
import numpy as np

high= np.log(data.sel(field="high"))
```

# How can you reduce slippage impace when trading?

Just apply some technique to reduce turnover:

```python
def get_lower_slippage(weights, rolling_time=6):
    return weights.rolling({"time": rolling_time}).max()

improved_weights = get_lower_slippage(weights, rolling_time=6)
```

# How to use technical analysis indicators?

For available indicators see the source code of the library: /qnt/ta

## ATR

```python
def get_atr(data, days=14):
    high = data.sel(field="high") * 1.0
    low  = data.sel(field="low") * 1.0
    close= data.sel(field="close") * 1.0

    return qnta.atr(high, low, close, days)

atr= get_atr(data, days=14)
```

## EMA

```python
prices= data.sel(field="high")
prices_ema= qnta.ema(prices, 15)
```

## TRIX

```python
prices= data.sel(field="high")
prices_trix= qnta.trix(prices, 15)
```

## ADL and EMA

```python
adl= qnta.ad_line(data.sel(field="close")) * 1.0
adl_ema= qnta.ema(adl, 18)
```

# How can you check the quality of your strategy?

```python
import qnt.output as qnout
qnout.check(weights, data, "stocks_nasdaq100")
```

or

```python
stat= qnstats.calc_stat(data, weights)
display(stat.to_pandas().tail())
```

or

```python
import qnt.graph   as qngraph
statistics= qnstats.calc_stat(data, weights)
display(statistics.to_pandas().tail())

performance= statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())
qnstats.print_correlation(weights, data)

```

# An example using pandas

One can work with pandas DataFrames at intermediate steps and at the end convert them to xarray data structures:

```python
def get_price_pct_change(prices):
    prices_pandas = prices.to_pandas()
    assets = data.coords["asset"].values
    for asset in assets:
        prices_pandas[asset] = prices_pandas[asset].pct_change()
    return prices_pandas

prices = data.sel(field="close") * 1.0
prices_pct_change = get_price_pct_change(prices).unstack().to_xarray()
```

# How to submit a strategy to the competition?

Check that weights are fine:

```python
import qnt.output as qnout
qnout.check(weights, data, "stocks_nasdaq100")
```

If everything is ok, write the weights to file:

```python
qnout.write(weights)
```

In your **personal account**:

* **choose** a strategy;
* click on the **Submit** button;
* select the type of competition.

At the beginning you will find the strategy under the **Checking** area:

* **Sent strategies** > **Checking**.

If technical checks are successful, the strategy will go under the **Candidates** area:

* **Sent strategies** > **Candidates**.

Otherwise it will be **Filtered**:

* **Sent strategies** > **Filtered**

and you should inspect error and warning messages.

Note that a strategy under the **Candidates** area should have a Sharpe ratio larger than 1 for being eligible for a prize. Please check warning messages in your **Candidates** area!